In [1]:
import sys
sys.path.insert(1,'../tools')
import pandas as pd
import numpy as np
import html
import random
from IPython.core.display import display, HTML
import utils
import matplotlib.pyplot as plt

In [2]:
from torch.utils.data import Dataset, DataLoader
from transformers import modeling_bert
import numpy as np
import torch
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
from tqdm import tqdm
import pdb
import torch.nn.functional as F
from torch import optim
import time
import imp
import os
from sklearn.metrics import confusion_matrix
import json
import sklearn

In [3]:
imp.reload(utils)
hpara1 = utils.hpara()
hpara1.ratio = 2
hpara1.batch_size = 4
hpara1.word_layers=4
hpara1.sent_layers=1
hpara1.use_position_embedding = False
hpara1.word_lr = 1e-04 #5e-05
hpara1.sent_lr = 8e-04 #2e-04
hpara1.decay_step = 10
hpara1.decay_gamma = 0.5
hpara1.max_sent_len = 64
hpara1.fix_cls = True
hpara1.head_num = 12
hpara1.att_decay_step=3
hpara1.att_decay_rate=0.5
hpara1.max_doc_len = 300

In [4]:
hpara1.use_SSI_Bert = False
hpara1.use_narrow = False

In [6]:
with open('./exp/best_clinical_bert/hpara.json', 'w') as fp:
    json.dump(hpara1.__dict__, fp)


In [7]:
if hpara1.use_SSI_Bert:
    pretrain_model_dir = '../BERT/bert/infection_bert_fix_gast'
else:
    pretrain_model_dir = '../BERT/pretrained_bert_tf/biobert_pretrain_output_all_notes_150000'
vocab_file = os.path.join(pretrain_model_dir,'vocab.txt')
tokenizer = utils._load_tf_tokenizer(vocab_file = vocab_file)
bert_config_file = os.path.join(pretrain_model_dir,'bert_config.json')

trained_word_model = './exp/best_clinical_bert/save_word_10.bin' 
trained_sent_model = './exp/best_clinical_bert/save_sent_10.bin' 

if hpara1.use_SSI_Bert and 'SSI' not in trained_word_model:
    print('ERROR')
if not hpara1.use_SSI_Bert and 'SSI'  in trained_word_model:
    print('ERROR')

In [8]:
config = modeling_bert.BertConfig.from_json_file(bert_config_file)
config.num_hidden_layers = hpara1.word_layers
config.output_attentions = True
model_word = modeling_bert.BertModel(config)
model_word.load_state_dict(torch.load(trained_word_model))
cls_weight = model_word.state_dict()['embeddings.word_embeddings.weight'][101]

In [9]:
config_doc = modeling_bert.BertConfig.from_json_file(bert_config_file)
config_doc.num_hidden_layers = hpara1.sent_layers
config_doc.output_attentions = True
config_doc.num_attention_heads = hpara1.head_num
use_position_embedding = hpara1.use_position_embedding
model_sent = modeling_bert.BertModel_no_embedding(config_doc,cls_weight,use_position_embedding=use_position_embedding)
model_sent.load_state_dict(torch.load(trained_sent_model))

<All keys matched successfully>

In [10]:
model_word = model_word.cuda()
model_sent = model_sent.cuda()

In [19]:
_,_,_,label_test = utils.load_raw_data_new()
svm_y_pred = np.load('../PU_classifier_training/svm_y_pred.npy')
bert_y_pred = np.load('y_hat_clinical_noN.npy')[-len(label_test):]
svm_thres = 0.37
bert_thres = 0.64
diff_svm = (svm_y_pred > svm_thres) - np.array(label_test)
diff_bert = (bert_y_pred > bert_thres) - np.array(label_test)
fn_review = set(np.where(diff_svm == -1)[0]).intersection(np.where(diff_bert == -1)[0])
print(fn_review)

{649, 529, 26, 27, 541, 285, 415, 419, 292, 298, 173, 562, 314, 699, 444, 75, 471, 88, 475, 226, 507}


In [9]:
# False positive examples
fp_review = {38, 44, 60, 71, 137, 270, 311, 329, 393, 449, 461, 513, 523, 592, 625, 662, 666, 705}

In [14]:
def write_key_sent(sent_weight,doc,write_file,index):
    log = 'Doc {}: <br />key sentence1: {}, score: {}, <br />key sentence2: {}, score: {}, <br />key sentence3: {}, score: {} <br />'
    key_sent_index = np.where(sent_weight[1:-1]>0.001)[0] + 1
    key_sent_index = key_sent_index[np.argsort(sent_weight[key_sent_index])].tolist()
    doc.append('SEP')
    doc.insert(0,'CLS')        
    #pdb.set_trace()
    if len(key_sent_index) == 0:
        key_sent_index = [np.argmax(sent_weight[:])]
    if len(key_sent_index) <3:
        [key_sent_index.append(key_sent_index[0]) for i in range(3 - len(key_sent_index))]

    sent1 = doc[key_sent_index[-1]]
    weight1 = sent_weight[key_sent_index[-1]]

    sent2 = doc[key_sent_index[-2]]
    weight2 = sent_weight[key_sent_index[-2]]

    sent3 = doc[key_sent_index[-3]]
    weight3 = sent_weight[key_sent_index[-3]]

    to_print = log.format(index,sent1,weight1,sent2,weight2,sent3,weight3)
    write_file.writelines(to_print)
    write_file.close()

In [20]:
import pickle
src_dir = '../preprocessing/result/ratio2/'
text_test  = pickle.load(open(src_dir+'merged_text_test.pkl','rb')) 
label_test = pickle.load(open(src_dir+'label_test.pkl','rb'))
##Change here!
indices_to_review = fn_review
text_to_review = []
label_to_review = []
for ind in indices_to_review:
    text_to_review.append(text_test[ind])
    label_to_review.append(label_test[ind])
params_all = {'batch_size': 1,
    'shuffle': False,
    'num_workers': 6}
review_set = utils.DocDataset(text_to_review, label_to_review)
review_generator = DataLoader(review_set, **params_all)

In [21]:
date = str(list(time.localtime())[0:3]).replace(', ','_')
save_dir = './statis/'+date[1:-1]+'/'

    
if not os.path.exists(save_dir):
    cmd = 'mkdir -p ' + save_dir
    os.system(cmd)
     
with open(save_dir + 'used_model.txt','w') as f:
    f.writelines(trained_word_model + '\n' +trained_sent_model)
    
with open(save_dir + 'hpara.json', 'w') as fp:
    json.dump(hpara1.__dict__, fp)
    
max_epoch = hpara1.max_epoch
log = 'Iter {}/{}, Loss={:.3f},accu={:.3f},time={:.3f}\n'
from tqdm import tqdm
batch_size = hpara1.batch_size
accumulation_steps = hpara1.accumulation_steps
max_sent_len = hpara1.max_sent_len
max_doc_len = hpara1.max_doc_len
#progress_bar = tqdm(enumerate(training_generator))
para_dict = {}
hpara_list = []

In [22]:
hpara1.max_doc_len

300

In [23]:
model_sent.eval()
model_word.eval()
pred_list = []
y_list = []
y_hat = []
total_num = 0
correct = 0
max_doc_len = hpara1.max_doc_len
batch_size = hpara1.batch_size
narrow = hpara1.use_narrow
def run_model(doc,label):
    label = label.cuda()
    batch_count=0
    sent_num = 0
    end_ind = 0
    input_tensors = torch.zeros([1,max_doc_len,hpara1.hidden_size]).cuda()
    while end_ind <= len(doc) and end_ind < max_doc_len-1:
        batch_sent = utils.batch_sent_loader(doc,batch_size,batch_count,max_doc_len=max_doc_len)
        cur_batch_size = len(batch_sent)
        sent_num += cur_batch_size
        input_ids = torch.zeros(cur_batch_size,max_sent_len).long().cuda()
        input_mask = torch.ones(cur_batch_size,max_sent_len).long().cuda()
        for i in range(len(batch_sent)):
            tmp_ids,tmp_mask= utils.word_tokenize(batch_sent[i][0],max_sent_len,tokenizer)
            input_ids[i,:] = torch.tensor(tmp_ids)
            input_mask[i,:] = torch.tensor(tmp_mask)
        _,tmp_input_tensors,word_att_output = model_word(input_ids,attention_mask=input_mask)
        start_ind = batch_count*batch_size+1
        end_ind = start_ind + cur_batch_size
        input_tensors[0,start_ind:end_ind] = tmp_input_tensors
        batch_count += 1

    sent_mask = [1]*(end_ind)
    while len(sent_mask)<max_doc_len:
        sent_mask.append(0)
    sent_mask = torch.tensor(sent_mask).unsqueeze(0).cuda()
    if narrow:
        _,proba,sent_att_output = model_sent(input_tensors,label,attention_mask=sent_mask,epoch=cur_epoch)
    else:
        _,proba,sent_att_output = model_sent(input_tensors,label,attention_mask=sent_mask)
    return sent_att_output,doc,proba,label

In [24]:
import pdb
max_alpha = 0.6 
colored_text = '<span style="background-color:rgba({},100,{},{} ); line-height:30px">{}</span>' 
highlighted_text = []
y_list = []
y_hat = []
proba = []
pred_list= []
fp_proba_list = []
thres = 0.5
count = 0

for doc_count,(doc,label) in enumerate(review_generator):
    sent_att_output,doc,proba,label = run_model(doc,label)
    proba = proba.cpu().detach().numpy()
    pred_list.append(proba[0,1]>thres)
    is_wrong = (proba[0,1]>thres) - label.cpu().numpy()
    sent_weight = utils.One_layer_back_trace(sent_att_output[-1])
    sent_weight = sent_weight[0]

    max_alpha = 0.1
    highlighted_text = []
    for i in range(min(299,len(doc))):
        sent = doc[i][0]
        weight = sent_weight[i+1]

        red_value =  weight/ max(sent_weight)*255
        blue_value = 255 - red_value
        color_intensity = max(max_alpha, weight/ max(sent_weight))
        new_colored_text = colored_text.format(red_value,blue_value,color_intensity,html.escape(sent+'\n'))
        highlighted_text.append(new_colored_text)
        #highlighted_text.append('<span style="background-color:rgba(135,206,250,' + str(weight / max_alpha) + ');">' + html_escape(sent+'\n') + '</span>')

    highlighted_text = '<br />'.join(highlighted_text)
    is_wrong = 1
    #display(HTML(highlighted_text))
    if (is_wrong == 1):
        count +=1
        write_file =  open(save_dir + str(count) + '.html','w')
        write_key_sent(sent_weight,doc,write_file,count)

        fp_proba_list.append(proba[0,1])
        print(count)
        with open(save_dir + str(count) + '.html','a') as f:
            f.writelines(highlighted_text)

        #print(proba)
        #pdb.set_trace()
        

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21


In [25]:
proba[0,1]

-0.5386936